# САМОСТОЯТЕЛЬНАЯ РАБОТА

В рамках данной самостоятельной работы будет закрепляться навык оценки эффективности внедрённой фичи. Ниже приведены задачи работы.

### Задачи:
1. Посчитать показатели Retention Rate седьмого дня и ARPPU за два периода: до и после внедрения фичи (май и июнь 2019 соответственно);
2. Сформировать выводы по анализу.

### Выполнение:

1. Перво-наперво загрузим требуемые библотеки и составим функции для получения наших данных.  

In [1]:
# Нам нужны библиотеки: pandas для работы с датафреймами, numpy для работы с датами, psycopg2 для работы с SQL запросами
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras

2. Получим данные для мая и июня 2019 года требуемые нам для проведения анализа:
- количество зарегистрированных пользователей;
- количество активных пользователей;
- доход.

In [2]:
def getRegisteredUsersCount():
    query = '''SELECT r.registration_date, count(distinct(r.user_id)) as registered_users_count
    FROM case9.registration r
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

registered_users_count = pd.DataFrame(getRegisteredUsersCount())

In [3]:
def getActiveUsersCountWithCohorts():
    query = '''SELECT ac.activity_date, r.registration_date, count(distinct(ac.user_id)) as active_users_count
    FROM case9.user_activity ac
    LEFT JOIN case9.registration r ON r.user_id = ac.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data 

active_users_count_with_cohorts = pd.DataFrame(getActiveUsersCountWithCohorts())

In [4]:
def getRevenue():
    query = '''SELECT re.date, r.registration_date, count(distinct(re.user_id)) as users_count_with_revenue, sum(re.revenue) as revenue
    FROM case9.user_revenue re
    LEFT JOIN case9.registration r ON r.user_id = re.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

revenue = pd.DataFrame(getRevenue())

3. Смотрим данные, по одной таблице из каждого варианта хватит для понимания что делать далее.

In [5]:
registered_users_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   registration_date       61 non-null     object
 1   registered_users_count  61 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


In [6]:
active_users_count_with_cohorts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1891 entries, 0 to 1890
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   activity_date       1891 non-null   object
 1   registration_date   1891 non-null   object
 2   active_users_count  1891 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 44.4+ KB


In [7]:
revenue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1745 entries, 0 to 1744
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      1745 non-null   object 
 1   registration_date         1745 non-null   object 
 2   users_count_with_revenue  1745 non-null   int64  
 3   revenue                   1745 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 54.7+ KB


4. Видим что во всех таблицах что величины связанные с датами имеют формат object, т.е. строковая величина. Необходимо в каждой таблице поменять формат дат на формат дат

In [8]:
registered_users_count['registration_date'] = pd.to_datetime(registered_users_count['registration_date'])

In [9]:
for col in ['activity_date','registration_date']:
    active_users_count_with_cohorts[col] = pd.to_datetime(active_users_count_with_cohorts[col])

In [10]:
revenue = revenue.rename(columns={'date':'activity_date'})

In [11]:
for col in ['activity_date','registration_date']:
    revenue[col] = pd.to_datetime(revenue[col])

5. Объединим наши таблицы в одну

In [12]:
users_with_revenue = active_users_count_with_cohorts.merge(registered_users_count, on = 'registration_date', how = 'left')
users_with_revenue = users_with_revenue.merge(revenue, on = ['registration_date','activity_date'], how = 'left')
users_with_revenue.head()

,activity_date,registration_date,active_users_count,registered_users_count,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,2590,6516,1672.0,2357.580
1,2019-05-02,2019-05-01,2146,6516,1420.0,514.679
2,2019-05-02,2019-05-02,2049,5309,1367.0,1629.140
3,2019-05-03,2019-05-01,1637,6516,1048.0,390.952
4,2019-05-03,2019-05-02,1817,5309,1164.0,533.608


In [13]:
users_with_revenue.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1891 entries, 0 to 1890
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   activity_date             1891 non-null   datetime64[ns]
 1   registration_date         1891 non-null   datetime64[ns]
 2   active_users_count        1891 non-null   int64         
 3   registered_users_count    1891 non-null   int64         
 4   users_count_with_revenue  1745 non-null   float64       
 5   revenue                   1745 non-null   float64       
dtypes: datetime64[ns](2), float64(2), int64(2)
memory usage: 103.4 KB


In [14]:
for col in ['revenue','users_count_with_revenue']:
    users_with_revenue[col] = users_with_revenue[col].fillna(0)

In [15]:
users_with_revenue.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1891 entries, 0 to 1890
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   activity_date             1891 non-null   datetime64[ns]
 1   registration_date         1891 non-null   datetime64[ns]
 2   active_users_count        1891 non-null   int64         
 3   registered_users_count    1891 non-null   int64         
 4   users_count_with_revenue  1891 non-null   float64       
 5   revenue                   1891 non-null   float64       
dtypes: datetime64[ns](2), float64(2), int64(2)
memory usage: 103.4 KB


6. Считаем когорты по дате регистрации

In [16]:
users_with_revenue['lifetime'] = users_with_revenue['activity_date'] - users_with_revenue['registration_date']
users_with_revenue['lifetime'] = users_with_revenue['lifetime']/np.timedelta64(1,'D')
users_with_revenue['lifetime'] = users_with_revenue['lifetime'].astype(int)
users_with_revenue.head()

,activity_date,registration_date,active_users_count,registered_users_count,users_count_with_revenue,revenue,lifetime
0,2019-05-01,2019-05-01,2590,6516,1672.0,2357.580,0
1,2019-05-02,2019-05-01,2146,6516,1420.0,514.679,1
2,2019-05-02,2019-05-02,2049,5309,1367.0,1629.140,0
3,2019-05-03,2019-05-01,1637,6516,1048.0,390.952,2
4,2019-05-03,2019-05-02,1817,5309,1164.0,533.608,1


7. Фильтруем по кагорте 7-го дня для каждого месяца по отдельности.

In [17]:
retention_table_7d_05_19 = users_with_revenue.loc[(users_with_revenue['lifetime'] == 7)&
                                                  (users_with_revenue['registration_date']>='2019-05-01')&
                                                  (users_with_revenue['registration_date']<'2019-06-01')]
retention_table_7d_05_19.head()

,activity_date,registration_date,active_users_count,registered_users_count,users_count_with_revenue,revenue,lifetime
28,2019-05-08,2019-05-01,953,6516,616.0,63.9065,7
37,2019-05-09,2019-05-02,745,5309,490.0,313.7230,7
47,2019-05-10,2019-05-03,895,6266,571.0,110.7260,7
58,2019-05-11,2019-05-04,1186,8639,750.0,124.8590,7
70,2019-05-12,2019-05-05,1358,9787,885.0,174.9240,7


In [18]:
retention_table_7d_06_19 = users_with_revenue.loc[(users_with_revenue['lifetime'] == 7)&
                                                  (users_with_revenue['registration_date']>='2019-06-01')&
                                                  (users_with_revenue['registration_date']<'2019-07-01')]
retention_table_7d_06_19.head()

,activity_date,registration_date,active_users_count,registered_users_count,users_count_with_revenue,revenue,lifetime
736,2019-06-08,2019-06-01,545,4833,365.0,176.3160,7
767,2019-06-09,2019-06-02,683,5255,461.0,193.8390,7
798,2019-06-10,2019-06-03,484,4193,320.0,27.7548,7
829,2019-06-11,2019-06-04,511,4194,320.0,54.9610,7
860,2019-06-12,2019-06-05,472,3998,301.0,63.1554,7


8. Рассчитаем коэффициент возвращаемости или Retention Rate для наших данных

In [19]:
retention_rate_7d_05_19 = round(retention_table_7d_05_19['active_users_count'].sum() / retention_table_7d_05_19['registered_users_count'].sum()*100, 2)
retention_rate_7d_05_19

12.93

In [20]:
retention_rate_7d_06_19 = round(retention_table_7d_06_19['active_users_count'].sum() / retention_table_7d_06_19['registered_users_count'].sum()*100, 2)
retention_rate_7d_06_19

12.56

По данный Retention Rate мы видим что в мае этот показатель немного выше чем в июне. Пока не будем делать каких-либо поспешных выводов. Смотрим ARPPU для наших когорт.

9. Считаем ARPPU

In [21]:
arppu_7d_05_19 = users_with_revenue['revenue'][users_with_revenue['registration_date']>="2019-05-01"][users_with_revenue['registration_date']<"2019-06-01"].sum() / users_with_revenue['users_count_with_revenue'][users_with_revenue['registration_date']>="2019-05-01"][users_with_revenue['registration_date']<"2019-06-01"].sum()
arppu_7d_05_19

0.3061245675177216

In [22]:
arppu_7d_06_19 = users_with_revenue['revenue'][users_with_revenue['registration_date']>="2019-06-01"][users_with_revenue['registration_date']<"2019-07-01"].sum() / users_with_revenue['users_count_with_revenue'][users_with_revenue['registration_date']>="2019-06-01"][users_with_revenue['registration_date']<"2019-07-01"].sum()
arppu_7d_06_19

0.3931940184296097

ARPPU для июня выше на 30% и составляет 0,39$

## Выводы:
- Retention Rate 7 дня с небольшим снижением, но остался примерно на прежнем уровне
- ARPPU в июне вырос на 30%

На текущий момент мы не можем сказать по какой причине это произошло, т.к. не имеем необходимых данных. На будущее можно настроить сбор данных и по данному направлению.